# Tweets URL Cleaner
Use this notebook to clean tweet files that have been created before the correct implementation of the URL cleaner was applied.

In [ ]:
import os
import json
import re

tweets_dir = "../lib/GetOldTweets-python/out/completed"
excluded = ["ipynb_checkpoints"]

def read_large_file(file_object):
    while True:
        data = file_object.readline()
        if not data:
            break
        yield data.rstrip('\n')

def remove_urls_from_text(text):
    patterns = [r'pic\.twitter\.com/\S*', r'https?://\S*']
    for p in patterns:
        text = re.sub(p, "", text).strip()
    return text

def should_skip(name):
    skip = False
    for el in excluded:
        if el in name:
            skip = True
            break
    return skip
        
# Clean hashtags file by removing additional URLs from tweet text
def clean_tweets():
    for tweets_filename in os.listdir(tweets_dir):
        if should_skip(tweets_filename):
            continue
        input_filepath = os.path.join(tweets_dir,tweets_filename)
        output_filepath = os.path.join(tweets_dir,os.path.splitext(tweets_filename)[0]+"_cleaned.json")
        print("\n%s" %tweets_filename)
        print("Occurrences of pattern 'pic\.twitter\.com/\S*':")
        !egrep "pic\.twitter\.com/\S*" $input_filepath | wc -l
        print("Occurrences of pattern 'https?://\S*':")
        !egrep "https?://\S*" $input_filepath | wc -l
        with open(output_filepath, "w") as fout:
            with open(input_filepath) as fin:
                for line in read_large_file(fin):
                    tweet = json.loads(line)
                    tweet["text"] = remove_urls_from_text(tweet["text"])
                    if "quoted_status" in tweet:
                        tweet["quoted_status"]["text"] = remove_urls_from_text(tweet["quoted_status"]["text"])
                    if "retweeted_status" in tweet:
                        tweet["retweeted_status"]["text"] = remove_urls_from_text(tweet["retweeted_status"]["text"])
                    newLine = json.dumps(tweet)
                    fout.write(newLine+"\n")
        print("Post-cleanup check:")
        print("Occurrences of pattern 'pic\.twitter\.com/\S*':")
        !egrep "pic\.twitter\.com/\S*" $output_filepath | wc -l
        print("Occurrences of pattern 'https?://\S*':")
        !egrep "https?://\S*" $output_filepath | wc -l

In [ ]:
clean_tweets()